# Project 2 

In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from preprocessing import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Main 

In [6]:
DATA_Ux_PATH = 'data/u1_very_small.csv'
DATA_Uy_PATH = 'data/u2_very_small.csv'
Ux = load_csv_data(DATA_Ux_PATH)
Uy = load_csv_data(DATA_Uy_PATH)

In [7]:
print(Ux.shape)

(137725, 110)


In [10]:
print(Ux.shape[0]/25)

5509.0


In [13]:
ratio_pts = 0.1
ratio_time = 1

new_Ux, new_Uy = sample(Ux, Uy, ratio_pts, ratio_time)
print(new_Ux.shape, new_Uy.shape )

(13750, 110) (13750, 110)


In [14]:
print((new_Ux.shape[0]/25)*(new_Ux.shape[1]/25))

2420.0


In [9]:
print(new_Ux.shape)

(13750, 55)


1. modeliser mathématiquement l’écoulement sanguin à l’aide de PDE dépendante de 2-3 paramètres physique
2. simuler l’écoulement par un code d’éléments fini ou similaire
3. générer beaucoup de solutions avec une grande nombre de paramètres différents. (Les 2-3, pris de façon aléatoire)
4. a. utiliser les solutions numérique pour établir un auto encoder qui au milieu n’ai que 2-5 hyper-paramètres libres  
b. quel erreur on obtient ? Est-ce possible de réduire le nombre d’hyper-paramètres ?
5. étudier s’il y a une rélation entre les 2-5 hyper-paramètre et les paramètres physique  
a. à l’aide de statistiques
5. b. à l’aide d’un DNN (différent de 4a)
6. (optionnel) faire un DNN entre l’input de 4a et output les paramêtres physique. Et/ou l’inverse.
7. Discussion et conclusions 

## 1. Data exploration

## 2. Feature processing

## 3. Solution generation

## 4. Auto-encoder

## 5. Hyper-parameters and physics parameters relationship

## 6. Discussion & conclusion